In [22]:
import pandas as pd

df = pd.read_csv('../data/5_full_dataset.csv')
hot100_df = pd.read_csv('../data/1_hot100_cleaned.csv')


In [23]:
# Create isHot column
df['isHot'] = df['title'].isin(hot100_df['title']).astype(int)

In [26]:
# drop Bru's genre columns
df = df.drop(columns=['rock', 'pop', 'hip-hop', 'country', 'latin', 'indie', 'folk', 'soul', 'punk', 'metal', 'jazz', 'classical', 'reggae', 'blues'])

In [28]:
df=df.drop(columns=['rap', 'dance'])

In [ ]:
display(df.head())

In [ ]:
import ast

# Combine all genres into one list
all_genres = []
for genre_list in df['genres'].dropna():  # skip NaN values
    genres = ast.literal_eval(genre_list)  # convert string to list
    all_genres.extend(genres)

# Create a list of unique genres
unique_genres = list(set(all_genres))

print(f"Total number of unique genres: {len(unique_genres)}")
print("\nFirst 10 genres as example:")
print(unique_genres[:10])

In [ ]:
# 1. Get all unique genres (no threshold filtering)
all_genres = []
for genre_list in df['genres'].dropna():
    genres = genre_list.split(', ')
    genres = [genre.strip() for genre in genres]
    all_genres.extend(genres)

unique_genres = sorted(list(set(all_genres)))  # Get all unique genres and sort them

# 2. Create one-hot encoding for all genres
def encode_all_genres(genre_string):
    if pd.isna(genre_string):
        return [0] * len(unique_genres)
    
    genre_list = genre_string.split(', ')
    return [1 if genre in genre_list else 0 for genre in unique_genres]

# Create encoded columns
genre_encoded = pd.DataFrame(
    df['genres'].apply(encode_all_genres).tolist(),
    columns=unique_genres
)

# Add encoded columns to original dataframe
df_spotify_songs_encoded_full = pd.concat([df, genre_encoded], axis=1)

# Verify the number of columns
print(f"Total number of columns: {len(df_spotify_songs_encoded_full.columns)}")
print(f"Number of genre columns: {len(unique_genres)}")

In [35]:
df_spotify_songs_encoded_full = df_spotify_songs_encoded_full.dropna(subset=['genres'])  # only drops rows where 'genres' is null

In [ ]:
df_spotify_songs_encoded_full.isnull().sum()

In [ ]:
# drop non-numerical columns
drop_columns = ['title', 'artist', 'album', 'popularity', 'is_explicit', 'isHot', 'album_cover', 'duration_seconds', 'genres']
df_numerical = df_spotify_songs_encoded_full.drop(columns= drop_columns).copy()

# Display the first few rows of the new dataframe
print("Shape of numerical dataframe:", df_numerical.shape)
df_numerical.head()

In [ ]:
# Clean up column names by removing quotes and brackets
df_numerical.columns = df_numerical.columns.str.replace('"', '').str.replace("'", '').str.replace('[', '').str.replace(']', '')

# Verify the cleaned column names
print("Cleaned column names:")
print(df_numerical.columns.tolist()[:20])  # Show first 10 columns as example

In [ ]:
# Save the numerical dataset to CSV
df_numerical.to_csv('../data/5_spotify_numerical.csv', index=False)
print("Dataset saved successfully!")

Dropping genres with less than 30 tracks

In [ ]:
# Read the dataset
df_spotify_numerical = pd.read_csv('../data/5_spotify_numerical.csv')

# Sum up each genre column (since they're one-hot encoded, sum equals count)
genre_counts = df_spotify_numerical.iloc[:, 9:].sum().sort_values(ascending=False)  # Skip first 9 non-genre columns

# Display the counts
print(f"Number of tracks per genre (total tracks: {len(df_spotify_numerical):,}):")
print("=" * 50)
for genre, count in genre_counts.items():
    if count > 0:  # Only show genres that have at least one track
        print(f"{genre}: {int(count):,} tracks")

In [ ]:
import re
# Combine versioned genre columns
def combine_versioned_columns(df):
    # Get all column names
    columns = df.columns.tolist()
    
    # Dictionary to store base genres and their versioned columns
    genre_versions = {}
    
    # Regular expression to match versioned columns (e.g., 'rock.1', 'blues.2')
    pattern = r'(.+?)\.?\d*$'
    
    # Group columns by their base genre name
    for col in columns:
        match = re.match(pattern, col)
        if match:
            base_genre = match.group(1)
            if base_genre not in genre_versions:
                genre_versions[base_genre] = []
            genre_versions[base_genre].append(col)
    
    # Create new columns combining the versions
    for base_genre, versioned_cols in genre_versions.items():
        if len(versioned_cols) > 1:  # Only process if there are multiple versions
            # Create new column with the maximum value across all versions
            df[base_genre] = df[versioned_cols].max(axis=1)
            # Drop the old versioned columns
            df = df.drop(columns=versioned_cols)
    
    return df

# Apply the function to combine versioned columns
df_spotify_numerical = combine_versioned_columns(df_spotify_numerical)

# Display the shape of the dataframe after combining columns
print("Shape of numerical dataframe after combining versions:", df_spotify_numerical.shape)
df_spotify_numerical.head()

#dpdate the csv file
df_spotify_numerical.to_csv('../data/5_spotify_numerical_combined.csv', index=False)


In [ ]:
# Read the dataset
df_spotify_numerical = pd.read_csv('../data/5_spotify_numerical_combined.csv')

# Sum up each genre column and create a DataFrame
genre_counts = df_spotify_numerical.iloc[:, 9:].sum().sort_values(ascending=False)
genre_df = pd.DataFrame({
    'genre': genre_counts.index,
    'track_count': genre_counts.values
})

# Add percentage column
total_tracks = len(df_spotify_numerical)
genre_df['percentage'] = (genre_df['track_count'] / total_tracks * 100).round(2)

# Filter out genres with zero tracks
genre_df = genre_df[genre_df['track_count'] > 0]

# Save to CSV
genre_df.to_csv('../data/genre_distribution.csv', index=False)

# Display first few rows
print(f"Total number of genres: {len(genre_df)}")
print("\nTop 10 genres by track count:")
print(genre_df.head(100))

In [ ]:
# Get genre columns (columns after the first 9 non-genre columns)
genre_columns = df_spotify_numerical.columns[9:]

# Calculate genre counts
genre_counts = df_spotify_numerical[genre_columns].sum()

# Get genres with at least 30 tracks
popular_genres = genre_counts[genre_counts >= 15].index

# Keep only non-genre columns and popular genre columns
columns_to_keep = list(df_spotify_numerical.columns[:9]) + list(popular_genres)
df_cleaned = df_spotify_numerical[columns_to_keep]

# Save the cleaned dataset
df_cleaned.to_csv('../data/5_spotify_numerical_cleaned.csv', index=False)

# Print statistics
print(f"Original number of genres: {len(genre_columns)}")
print(f"Number of genres with ≥30 tracks: {len(popular_genres)}")
print(f"Number of genres removed: {len(genre_columns) - len(popular_genres)}")
print(f"\nShape of new dataset: {df_cleaned.shape}")

# Display distribution of remaining genres
genre_dist = df_cleaned[popular_genres].sum().sort_values(ascending=False)
print("\nTop 10 genres in cleaned dataset:")
display(genre_dist)
